In [1]:
#import the random module
import random

#import pandas for df management
import pandas as pd

# import numpy
import numpy as np

#import matplotlib for plotting funtionality
import matplotlib.pyplot as plt

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# import citipy to get closest city from lat long
from citipy import citipy

# import the datetime module from the datetime library
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Import the requests library.
import requests

import os

In [2]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [3]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

602

In [4]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    try:
        city_rain = city_weather["rain"]["1h"]
        
    except:
        city_rain = 0.0 
        print("No Rainfall")

    try:
        city_snow = city_weather["snow"]["1h"]
        
    except:
        city_snow = 0.0
        print("No Snowfall")

    city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Rainfall(1h)":city_rain,
                          "Snowfall(1h)":city_snow})

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

ing Record 48 of Set 6 | mildura
No Rainfall
No Snowfall
Processing Record 49 of Set 6 | keetmanshoop
No Rainfall
No Snowfall
Processing Record 50 of Set 6 | korem
No Snowfall
Processing Record 1 of Set 7 | bereda
No Rainfall
No Snowfall
Processing Record 2 of Set 7 | heers
No Rainfall
No Snowfall
Processing Record 3 of Set 7 | edd
No Rainfall
No Snowfall
Processing Record 4 of Set 7 | namibe
No Rainfall
No Snowfall
Processing Record 5 of Set 7 | nguiu
City not found. Skipping...
No Rainfall
No Snowfall
Processing Record 6 of Set 7 | chagda
City not found. Skipping...
No Rainfall
No Snowfall
Processing Record 7 of Set 7 | felanitx
No Rainfall
No Snowfall
Processing Record 8 of Set 7 | bengkulu
No Snowfall
Processing Record 9 of Set 7 | nizhniy tsasuchey
No Rainfall
No Snowfall
Processing Record 10 of Set 7 | karratha
No Rainfall
No Snowfall
Processing Record 11 of Set 7 | hithadhoo
No Snowfall
Processing Record 12 of Set 7 | tiarei
No Rainfall
No Snowfall
Processing Record 13 of Set 7 

In [5]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Rainfall(1h),Snowfall(1h)
0,Lewiston,44.10,-70.21,73.99,93,75,23.04,US,2020-07-08 21:55:56,8.13,0
1,Marzuq,14.40,46.47,85.98,30,86,5.79,YE,2020-07-08 21:55:26,0.00,0
2,Tuktoyaktuk,69.45,-133.04,37.40,93,90,8.05,CA,2020-07-08 21:55:56,0.00,0
3,Rungata,69.45,-133.04,37.40,93,90,8.05,CA,2020-07-08 21:55:56,0.00,0
4,Jiangkou,25.49,119.20,82.40,83,100,17.90,CN,2020-07-08 21:55:56,0.26,0


In [6]:
# Create the output file (CSV).
output_data_file = os.path.join("Data","WeatherPy_Database.csv")

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")